# DTSC-670 Foundations of Machine Learning Models

## Training Linear Models

# Setup

This project requires Python 3.7 or above:

In [ ]:
import sys

assert sys.version_info >= (3, 7)

It also requires Scikit-Learn ≥ 1.0.1:

In [ ]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

As we did in previous chapters, let's define the default font sizes to make the figures prettier:

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# Linear Regression

## The Normal Equation

Let's start by creating some random data and plotting it.

In [ ]:
import numpy as np

np.random.seed(42)  # to make this code example reproducible
m = 100  # number of instances
X = 2 * np.random.rand(m, 1)  # column vector
y = 4 + 3 * X + np.random.randn(m, 1)  # column vector

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(X, y, "b.")
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.axis([0, 2, 0, 15])
plt.grid()
plt.show()

As a reminder, the ___Normal Equation___ is a closed form solution that yields the value of $ \theta $ that minimizes the cost function:

$$ \hat{\mathbf{\theta}} = (\mathbf{X}^T\mathbf{X})^{-1} \mathbf{X}^T \mathbf{y} $$

Perform linear regression using the normal equation:

In [ ]:
from sklearn.preprocessing import add_dummy_feature

X_b = add_dummy_feature(X)  # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y # '@' performs matrix multiplication

In [ ]:
theta_best

The function that we used to generate the data is y = 4 + 3x + Gaussian noise.  We would have hoped for θ<sub>0</sub> = 4 and θ<sub>1</sub> = 3 instead of θ<sub>0</sub> = 4.215 and θ<sub>1</sub> = 2.770. Close enough, but the noise made it impossible to recover the exact parameters of the original function.

The next two blocks of code makes predictions using $\hat{θ}$ and plots the model's predictions.

In [ ]:
X_new = np.array([[0], [2]])
X_new_b = add_dummy_feature(X_new)  # add x0 = 1 to each instance
y_predict = X_new_b @ theta_best
y_predict

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))  # extra code – not needed, just formatting
plt.plot(X_new, y_predict, "r-", label="Predictions")
plt.plot(X, y, "b.")

# extra code – beautifies and saves Figure 4–2
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.axis([0, 2, 0, 15])
plt.grid()
plt.legend(loc="upper left")

plt.show()

As usual, sklearn allows us to abstract away the math and code going on behind the scenes of the algorithm, and we are able to implement a linear regression model easily.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)

In [ ]:
# Scikit-Learn separates the bias term (intercept_) from the feature weights (coef_)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
lin_reg.predict(X_new)

Next, we will look at a very different way to train a linear regression model, which is better suited for cases where there are a large number of features or too many training instances to fit in memory.

# Gradient Descent
## Batch Gradient Descent

In [ ]:
eta = 0.1  # learning rate
n_epochs = 1000
m = len(X_b)  # number of instances

np.random.seed(42)
theta = np.random.randn(2, 1)  # randomly initialized model parameters

for epoch in range(n_epochs):
    gradients = 2 / m * X_b.T @ (X_b @ theta - y)
    theta = theta - eta * gradients

The trained model parameters:

In [ ]:
theta

That’s exactly what the Normal Equation found! 

## Stochastic Gradient Descent

In [ ]:
from sklearn.linear_model import SGDRegressor # remember that we used SGDClassifier in the classification chapter

# max_iter = max number of epochs
# tol = stopping criteria
# penalty = regularization term (we'll look at this later)
# eta0 = initial learning rate
# n_iter_no_change = number of iterations without improvement before stopping

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-5, penalty=None, eta0=0.01,
                       n_iter_no_change=100, random_state=42)
sgd_reg.fit(X, y.ravel())  # y .ravel() because fit() expects 1D targets


In [ ]:
y.ravel().ndim

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_

Notice that this comes close to the Batch Gradient Descent values but Stochastic Gradient Descent would be much faster with a larger dataset.

We won't cover mini-batch gradient descent, but you can check out the textbook for code that you can adjust to create your own version of mini-batch gradient descent.

# Polynomial Regression

What if your data is more complex than a straight line? Surprisingly, you can use a linear model to fit nonlinear data. A simple way to do this is to add powers of each feature as new features, then train a linear model on this extended set of features. This technique is called `polynomial regression`.

Note:  This is still a linear model because the coefficients are linear.

In [ ]:
### Generate nonlinear data based on simple quadratic equation
### y = ax^2 + bx + c + random noise

np.random.seed(42)
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X ** 2 + X + 2 + np.random.randn(m, 1)

In [ ]:
# plot the data

plt.figure(figsize=(6, 4))
plt.plot(X, y, "b.")
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.axis([-3, 3, 0, 10])
plt.grid()
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# instantiate a PolynomialFeatures class
poly_features = PolynomialFeatures(degree=2, include_bias=False)

# transform our features
X_poly = poly_features.fit_transform(X)

In [ ]:
# original feature
X[0]

In [ ]:
# original feature and it's square
X_poly[0]

In [ ]:
# instantiate a LinearRegression class
lin_reg = LinearRegression()

# fit the model using the polynomial transformed features
lin_reg.fit(X_poly, y)

In [ ]:
lin_reg.intercept_, lin_reg.coef_

Not bad considering the original function was: $$y = 0.5x_{1}^2 + 1x_{1} + 2 + noise$$.

In [ ]:
# extra code – this cell generates Figure 4–13

X_new = np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = poly_features.transform(X_new)
y_new = lin_reg.predict(X_new_poly)

plt.figure(figsize=(6, 4))
plt.plot(X, y, "b.")
plt.plot(X_new, y_new, "r-", linewidth=2, label="Predictions")
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.legend(loc="upper left")
plt.axis([-3, 3, 0, 10])
plt.grid()
plt.show()

Polynomial Regression is also capable of finding relationships between features, which a normal linear regression model is not able to do.  This is because `PolynomialFeatures` also add all combinations of features up to the given degree.  For example, if there were two features "a" and "b", `PolynomialFeatures` with degree=3 would not only add the features $a^2$, $a^3$, $b^2$, and $b^3$, but also the combinations $ab$, $a^2b$, and $ab^2$.

Be careful with polynomial regression for two reasons:  1) the increase of the degree of the polynomial causes an explosion of the number of features, and 2) using high-degree polynomials can severely overfit your training data.

In [ ]:
# extra code – this cell generates Figure 4–14

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

plt.figure(figsize=(6, 4))

for style, width, degree in (("r-+", 2, 1), ("b--", 2, 2), ("g-", 1, 300)):
    polybig_features = PolynomialFeatures(degree=degree, include_bias=False)
    std_scaler = StandardScaler()
    lin_reg = LinearRegression()
    polynomial_regression = make_pipeline(polybig_features, std_scaler, lin_reg)
    polynomial_regression.fit(X, y)
    y_newbig = polynomial_regression.predict(X_new)
    label = f"{degree} degree{'s' if degree > 1 else ''}"
    plt.plot(X_new, y_newbig, style, label=label, linewidth=width)

plt.plot(X, y, "b.", linewidth=3)
plt.legend(loc="upper left")
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.axis([-3, 3, 0, 10])
plt.grid()
plt.show()

So how do you choose the correct number of degrees?
- cross validation to determine the best RMSE
- using sklearn's `learning_curve()` function (see your textbook for more information)
- compare the Bayes Information Criteria (BIC) scores

# Regularized Linear Models

## Ridge Regression

Let's generate a very small and noisy linear dataset:

In [ ]:
# extra code – we've done this type of generation several times before
np.random.seed(42)
m = 20
X = 3 * np.random.rand(m, 1)
y = 1 + 0.5 * X + np.random.randn(m, 1) / 1.5
X_new = np.linspace(0, 3, 100).reshape(100, 1)

In [ ]:
# extra code – a quick peek at the dataset we just generated
plt.figure(figsize=(6, 4))
plt.plot(X, y, ".")
plt.xlabel("$x_1$")
plt.ylabel("$y$  ", rotation=0)
plt.axis([0, 3, 0, 3.5])
plt.grid()
plt.show()

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.1, solver="cholesky") # uses a matrix factorization technique by André-Louis Cholesky
ridge_reg.fit(X, y)

In [ ]:
# extra code – this cell generates Figure 4–17

def plot_model(model_class, polynomial, alphas, **model_kwargs):
    plt.plot(X, y, "b.", linewidth=3)
    for alpha, style in zip(alphas, ("b:", "g--", "r-")):
        if alpha > 0:
            model = model_class(alpha, **model_kwargs)
        else:
            model = LinearRegression()
        if polynomial:
            model = make_pipeline(
                PolynomialFeatures(degree=10, include_bias=False),
                StandardScaler(),
                model)
        model.fit(X, y)
        y_new_regul = model.predict(X_new)
        plt.plot(X_new, y_new_regul, style, linewidth=2,
                 label=fr"$\alpha = {alpha}$")
    plt.legend(loc="upper left")
    plt.xlabel("$x_1$")
    plt.axis([0, 3, 0, 3.5])
    plt.grid()

plt.figure(figsize=(9, 3.5))
plt.subplot(121)
plot_model(Ridge, polynomial=False, alphas=(0, 10, 100), random_state=42)
plt.ylabel("$y$  ", rotation=0)
plt.subplot(122)
plot_model(Ridge, polynomial=True, alphas=(0, 10**-5, 1), random_state=42)
plt.gca().axes.yaxis.set_ticklabels([])
plt.show()

*Figure 4-17. A linear model (left) and a polynomial model (right), both with various levels of Ridge regularization.  Note how
increasing `α` leads to flatter (i.e., less extreme, more reasonable) predictions, thus reducing the model’s variance but increasing its bias.*

## Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.1)

lasso_reg.fit(X, y)

In [ ]:
# extra code – this cell generates Figure 4–18
plt.figure(figsize=(9, 3.5))
plt.subplot(121)
plot_model(Lasso, polynomial=False, alphas=(0, 0.1, 1), random_state=42)
plt.ylabel("$y$  ", rotation=0)
plt.subplot(122)
plot_model(Lasso, polynomial=True, alphas=(0, 1e-2, 1), random_state=42)
plt.gca().axes.yaxis.set_ticklabels([])
plt.show()

## Elastic Net

The regularization term is a weighted sum of both ridge and lasso’s regularization terms, and you can control the mix ratio r. When r = 0, elastic net is equivalent to ridge regression, and when r = 1, it is equivalent to lasso regression

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)

elastic_net.fit(X, y)

Good work on the content in this module!  We have focused more on a high-level understanding of the material than what is covered in the textbook.  As you have time, I would suggest that you read through the entire chapter for a deeper understanding of the material.  There are also a few additional topics that we don't have the time to cover in this class but that will be helpful in your career.